In [1]:
import pandas as pd 
from fkscore import fkscore

In [2]:
data = 'Resources/scrubbed.csv'
df = pd.read_csv(data)

In [3]:
df_comments = df[['comments']]
df_comments['score'] = ''

In [4]:
text = df_comments.iloc[0,0]
print(text)

This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit


In [6]:
f = fkscore(text)
print(f.stats)
print(f.score)

{'num_sentences': 3, 'num_words': 26, 'num_syllables': 33}
{'readability': 90.661, 'grade': '5th Grade'}


In [5]:
df_comments.head()

,comments,score
0,This event took place in early fall around 194...,
1,1949 Lackland AFB&#44 TX. Lights racing acros...,
2,Green/Orange circular disc over Chester&#44 En...,
3,My older brother and twin sister were leaving ...,
4,AS a Marine 1st Lt. flying an FJ4B fighter/att...,
